In [1]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
%matplotlib notebook

# #1

In [14]:
def secondOrderPoly(x,pars):
    a,b,c = pars
    return a*x**2 + b*x + c
def lorentz(x,pars):
    return 1/(1-x**2)
def gaussian(x,pars):
    return np.exp(-0.5*x**2)

def simple_integrate(fun,pars,a,b,tol, total_eval):
    x=np.linspace(a,b,5)
    dx=(b-a)/(5-1)
    y=fun(x,pars)
    neval=len(x) #let's keep track of function evaluations
    f1=(y[0]+4*y[2]+y[4])/6.0*(b-a)
    f2=(y[0]+4*y[1]+2*y[2]+4*y[3]+y[4])/12.0*(b-a)
    myerr=np.abs(f2-f1)
    #print([a,b,f1,f2])
    total_eval += 1
    if (myerr<tol) or total_eval > 10:
        #return (f2)/1.0,myerr,neval
        return (16.0*f2-f1)/15.0,myerr,neval
    else:
        mid=0.5*(b+a)
        f_left,err_left,neval_left=simple_integrate(fun,pars,a,mid,tol/2.0 , total_eval)
        f_right,err_right,neval_right=simple_integrate(fun,pars,mid,b,tol/2.0 , total_eval)
        neval=neval+neval_left+neval_right
        f=f_left+f_right
        err=err_left+err_right
        return f,err,neval

def tracker_integrate(fun,pars,a,b,tol, total_recur_calls , track_x=[],track_y=[]):
    
    x=np.linspace(a,b,5); y = np.empty(len(x))
    dx=(b-a)/(5-1)
    neval = 0
    for i in range(len(x)):
        if np.any(x[i]==np.array(track_x)):
            y[i] = track_y[track_x.index(x[i])]
        else:
            #print('eval done')
            y[i] = fun(x[i],pars); neval += 1
            track_x.append(x[i]);  track_y.append(y[i])
            
    f1=(y[0]+4*y[2]+y[4])/6.0*(b-a)
    f2=(y[0]+4*y[1]+2*y[2]+4*y[3]+y[4])/12.0*(b-a)
    myerr=np.abs(f2-f1)
    #print([a,b,f1,f2])
    total_recur_calls += 1
    
    if (myerr<tol) or total_recur_calls > 10:
        return (16.0*f2-f1)/15.0,myerr,neval
    else:
        mid=0.5*(b+a)
        f_left , err_left , neval_left = tracker_integrate(fun,pars,a,mid,tol/2.0 , total_recur_calls , track_x,track_y)
        f_right , err_right , neval_right = tracker_integrate(fun,pars,mid,b,tol/2.0 , total_recur_calls , track_x,track_y)
        neval += neval + neval_left + neval_right
        f = f_left + f_right
        err = err_left + err_right
        return f , err  ,neval



In [15]:
a=2 ; b=3 ; c=4
pars = (a,b,c)
x_left=-5 ; x_right=5
tol = 10**-6

func = secondOrderPoly
f , err , neval = simple_integrate( func,pars , x_left,x_right , tol,0 )
F , ERR , NEVAL = tracker_integrate( func,pars , x_left,x_right , tol,0 , track_x=[],track_y=[])
print('Function evals for 2nd order poly using naive approach:',neval)
print('Function evals for 2nd order poly using bestest approach:',NEVAL)

func = lorentz
f , err , neval = simple_integrate( func,pars , x_left,x_right , tol,0 )
F , ERR , NEVAL = tracker_integrate( func,pars , x_left,x_right , tol,0 , track_x=[],track_y=[])
print('Function evals for lorentzian using naive approach:',neval)
print('Function evals for lorentzian using bestest approach:',NEVAL)

func = gaussian
f , err , neval = simple_integrate( func,pars , x_left,x_right , tol,0 )
F , ERR , NEVAL = tracker_integrate( func,pars , x_left,x_right , tol,0 , track_x=[],track_y=[])
print('Function evals for gaussian using naive approach:',neval)
print('Function evals for gaussian using bestest approach:',NEVAL)

Function evals for 2nd order poly using naive approach: 5
Function evals for 2nd order poly using bestest approach: 5
Function evals for lorentzian using naive approach: 2815
Function evals for lorentzian using bestest approach: 1694
Function evals for gaussian using naive approach: 795
Function evals for gaussian using bestest approach: 482


# #2

## **a)**

In [ ]:
# taken from cheb_expand.py from prof Sievers
def cheb_mat_uniform( nx , orda ):
    x = np.linspace(-1,1,nx)
    mat=np.zeros([nx,orda+1])
    mat[:,0]=1.0
    if orda>0:
        mat[:,1]=x
    if orda>1:
        for i in range(1,orda):
            mat[:,i+1]=2*x*mat[:,i]-mat[:,i-1]
    return mat

def get_cheby_coeffs(mat,y,keep):
    

def map_to_chebboundries(x):
    cheby_min = -1
    cheby_max = 1
    a = (cheby_max - cheby_min)/(x.max() - x.min())
    b = cheby_min - a*x.min()
    x_mapped = a*x + b
    return x_mapped


In [ ]:
xxx = np.linspace(0.5,1,10)
print(xxx)
print(map_to_chebboundries(xxx))
print(np.linspace(-1,1,10))

In [ ]:
n=1000
    # 'worse_accuracy' will track the highest error in our fit
orda = 10;   worse_accuracy = 1;   ncoeff = 1
x_left = 0.5 ;   x_right = 1

    # Since we are using a linspace, rescaling the values to (-1,1) can be done
    # simply by retaining the length of the original array and creating a np.linspace
    # with it and the correct cheby bounds
mat=cheb_mat_uniform(n,orda)
x = np.linspace(x_left,x_right,n)
y=np.log2(x)
    
    # Taken from Prof. Sievers' 'cheb_expand.py'
    # Performs least-squares fit
lhs=np.dot(mat.transpose(),mat)
rhs=np.dot(mat.transpose(),y)
fitp=np.dot(np.linalg.inv(lhs),rhs)

    # 
while worse_accuracy >= 10**-6 and ncoeff <= orda+1:
    y_cheb=np.dot(mat[:,:ncoeff],fitp[:ncoeff])
    worse_accuracy = np.max(np.abs(y_cheb-y))
    ncoeff += 1
ncoeff -= 1

    # Need to supply ncoeff-1 to this routine since it asks for the max degree and not the number of coefficients
leg_co = np.polynomial.legendre.legfit(x,y,ncoeff-1)    
y_leg = np.polynomial.legendre.legval(x,leg_co)

print( 'For '+str(orda)+'th order cheby truncated after',ncoeff,'terms, rms error is ',np.sqrt(np.mean((y_cheb-y)**2)),' , max error ',worse_accuracy )
print( 'For '+str(ncoeff-1)+'th order Legendre polyfit,               rms error is ',np.sqrt(np.mean((y_leg-y)**2)),' , max error ',np.max(np.abs(y_leg-y)) )
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.plot(x,y_cheb); plt.plot(x,y,c='black'); plt.plot(x,y_leg)
plt.subplot(1,2,2); plt.plot(x,y_cheb-y); plt.plot(x,y_leg-y); plt.axhline(0,c='black')
plt.legend(['Chebyshev Residual','Legendre Residual'])

**We could observe that a truncation including 8 terms in the Chebyshev series gives the desired maximum error in our fit. We therefore need at least a 7th order polynomial to obtain the desired maximum error of 10^-6. Plotting a Legendre polynomial fit using only 8 terms as well, we observe that at the edges the Cheby fit is better, while for most of the interval (in the center) the Legendre polynomial fit does best.**

# #3